# Analyze spectral difference between movement presence and absence

In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from collections import namedtuple
from itertools import compress, product
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version
from scipy import signal, stats

# import datetime as dt
# #mne
# import mne_bids
# import mne


In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')


In [ ]:
os.chdir(codepath)

# own functions
import lfpecog_features.moveDetection_preprocess as movePrep
import lfpecog_features.moveDetection_run as run_tap_detect
import lfpecog_features.moveDetection_pausedTapFinder as findTap

import lfpecog_analysis.load_SSD_features as load_ssd_fts
import lfpecog_analysis.get_acc_task_derivs as accDerivs

import utils.utils_fileManagement as utilsFiles
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_plotting.plotHelpers as plotHelp

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt_version)
# Python sys 3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]
# pandas 1.4.4
# numpy 1.23.3
# matplotlib 3.5.3

# STATS developed with:
# Python sys 3.10.8 (main, Nov  4 2022, 13:42:51) [MSC v.1916 64 bit (AMD64)]
# pandas 1.5.3
# numpy 1.26.2
# matplotlib 3.8.2

### 0. Define available Subjects


In [ ]:
import lfpecog_analysis.movement_psd_analysis as movePSD
import lfpecog_features.feats_helper_funcs as ftHelpers
import lfpecog_analysis.prep_movement_psd_analysis as prepMovePSD

import lfpecog_features.get_ssd_data as ssd
import lfpecog_analysis.ft_processing_helpers as ftProc


In [ ]:
FT_VERSION='v6'
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION=FT_VERSION)

main_data_path = os.path.join(get_project_path('data'),
                              'merged_sub_data',
                              SETTINGS["DATA_VERSION"])

SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=SETTINGS["DATA_VERSION"],
                                     FT_VERSION=FT_VERSION)
print(len(SUBS))


## 1. Spectral analysis - PSDs per movement-task-dyskinesia


- To Do:
    - plot unilat-dyskinesia in IPSI and CONTRA hemisphere (STN and ECoG)

In [ ]:
sub = '019'

sub_data_path = os.path.join(get_project_path('data'),
                             'merged_sub_data',
                             SETTINGS["DATA_VERSION"],
                             f'sub-{sub}')
fname = (f'{sub}_mergedData_{SETTINGS["DATA_VERSION"]}'
        '_acc_left.P')  # side does not matter for already detected bool labels

# load Acc-detected movement labels
accl = load_class_pickle(os.path.join(sub_data_path, fname))
accl = correct_acc_class(accl)

fname = (f'{sub}_mergedData_{SETTINGS["DATA_VERSION"]}'
        '_acc_right.P')  # side does not matter for already detected bool labels

# load Acc-detected movement labels
accr = load_class_pickle(os.path.join(sub_data_path, fname))
accr = correct_acc_class(accr)



In [ ]:
print(accl.data.shape, accl.fs, accl.colnames)
print(accr.data.shape, accr.fs, accr.colnames)

In [ ]:

# call from feats_extract_multivar.py
ssd_sub = ssd.get_subject_SSDs(
    sub=sub,
    incl_stn=True,
    incl_ecog=False,
    ft_setting_fname='ftExtr_spectral_v6.json',)

In [ ]:
# # get boolean arrays for movement labels corresponding to ephys
# ephys_arr = ssd_sub.lfp_left.lo_beta
# ephys_wintimes = np.array(ssd_sub.lfp_left.times)
# ephys_fs = ssd_sub.lfp_left.fs
# WINLEN_SEC = 10

# # create timestamps for every ephys sample in 2d array (2048 Hz)
# ephys_times2d = np.array([
#     np.arange(t, t + WINLEN_SEC, 1 / ephys_fs)
#     for t in ephys_wintimes
# ])
# nan_arr = np.isnan(ephys_arr)

# # get movement bools based on acc data (512 Hz)
# MOVE_BOOLS = {'no_move': accl.data[:, -1],
#               'any_move': np.sum(accl.data[:, -5:-1], axis=1) > 0,
#               'left_tap': accl.data[:, -5],
#               'left_allmove': (accl.data[:, -5] + accl.data[:, -3]) > 0,
#               'right_tap': accl.data[:, -4],
#               'right_allmove': (accl.data[:, -4] + accl.data[:, -2]) > 0}
# TASK = accl.data[:, 4]

In [ ]:
# CAT = False
# cdrs = ftProc.find_select_nearest_CDRS_for_ephys(
#     sub=sub, side='bilat',  
#     ft_times=ephys_wintimes / 60,
#     INCL_CORE_CDRS=True,
#     cdrs_rater='Patricia',
# )
 
# if CAT:
#     cdrs = ftProc.categorical_CDRS(cdrs,
#                             cutoff_mildModerate=3.5,
#                             cutoff_moderateSevere=7.5,)

In [ ]:
# importlib.reload(prepMovePSD)

# cdrs = ftProc.find_select_nearest_CDRS_for_ephys(
#     sub=sub, side='bilat',  
#     ft_times=ephys_wintimes / 60,
#     INCL_CORE_CDRS=True,
#     cdrs_rater='Patricia',
# )
# # cdrs = ftProc.categorical_CDRS(cdrs,
# #                             cutoff_mildModerate=3.5,
# #                             cutoff_moderateSevere=7.5,)

# mask_times = prepMovePSD.get_mask_timings(
#     orig_labels=cdrs,
#     orig_times=ephys_wintimes,
#     MASK='LID')

# # currently codes: rest=0, tap=1, free=2
# lid_mask = prepMovePSD.create_ephys_mask(
#     ephys_time_arr=ephys_times2d,
#     ephys_win_times=ephys_wintimes,
#     mask_times=mask_times,
#     MASK='LID',
# )

In [ ]:
import lfpecog_analysis.psd_analysis_classes as psdClass
import lfpecog_analysis.specific_ephys_selection as ephySel

import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_ssd_PSDs
import lfpecog_plotting.plot_move_spec_psd as plot_specPsd
import lfpecog_analysis.prep_stats_movLidspecPsd as prep_specStats
import lfpecog_analysis.psd_lid_stats as psd_Stats


In [ ]:
importlib.reload(ephySel)
importlib.reload(prepMovePSD)
importlib.reload(psdClass)
importlib.reload(ssd)

# # get single subClass (incl original 3d data, always creates new)
# test = psdClass.PSD_vs_Move_sub(sub='014')

# use existing
sub = '017'
data_path = 'D://Research/CHARITE/projects/dyskinesia_neurophys/data/'
picklepath = os.path.join(
    data_path,
    'windowed_data_classes_10s_0.5overlap',
    'selected_ephys_classes_all'
)
picklename = f'ephys_selections_{sub}.P'

sub_class = load_class_pickle(
    file_to_load=os.path.join(picklepath,
                                picklename),
    convert_float_np64=True
)

Check MASKs with ACC comparison

In [ ]:
importlib.reload(prepMovePSD)

move_masks, task_mask, lid_masks = prepMovePSD.create_ephys_masks(sub='012', verbose=True)

In [ ]:
for MOV_TYPE in move_masks.keys():
    match_sum = np.sum(np.logical_and(
        move_masks[MOV_TYPE] == 1,
        task_mask == 2
    ))
    print(f'{MOV_TYPE} during FREE: {match_sum / 2048} seconds')


In [ ]:
accl = accDerivs.get_raw_acc_traces(sub='012', side='left', data_version='v4.0')
print(accl.colnames)

#### Check and Plot extracted PSD data

Add STATS for PSD plotting

In [ ]:
# CREATE AND/OR PLOT SIGNIFICANCIES PER SELECTION (binary / linear)
importlib.reload(psd_Stats)
importlib.reload(psdClass)
importlib.reload(plot_specPsd)
importlib.reload(prep_specStats)

STATS_VERSION = '2Hz'

# prep_specStats.get_stats_REST_psds(
#     STAT_DATA_EXT_PATH = True,
#     STAT_LID_COMPARE = 'categs',
#     STATS_VERSION=STATS_VERSION,
#     ALPHA=.01,
#     PLOT_STATS=True,
#     MERGE_STNs=True,
#     STAT_PER_LID_CAT=True,
# )



prep_specStats.get_stats_MOVE_psds(
    STAT_DATA_EXT_PATH=True,
    STAT_LID_COMPARE='categs',
    STATS_VERSION=STATS_VERSION,
    PLOT_STATS=True,
    STAT_PER_LID_CAT=True,
    SKIP_MOVES=['FREEMOVE', 'FREENOMOVE'],  # FREEMOVE; FREENOMOVE; TAP; INVOLUNT
    MERGE_DYSK_SIDES=True,
    ALPHA=.01,
    ALT_BASELINE=False,
    REST_BASELINE=True,
    REST_u30_BASE=True,
)


In [ ]:
FIG_PATH = os.path.join(get_project_path('figures'),
                        'ft_exploration',
                        'data_v4.0_ft_v6',
                        'PSDs_state_specific')

cond_colors = {
    'nolid': 'green',
    'nolidbelow30': 'limegreen',
    'nolidover30': 'darkgreen',
    'alllid': 'blue', 'mildlid': 'orange',
    'moderatelid': 'red', 'severelid': 'purple',
}

Import 1-sec PSDs (rest-tap-dysk)


In [ ]:
importlib.reload(ephySel)
importlib.reload(psdClass)

RETURN_PSD_1sec = True

(
    PSDs_1s, BLs_1s
) = psdClass.get_allSpecStates_Psds(
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    incl_free=False,
    FEATURE='POWER',
)


Import 1-sec PSDs (free)

In [ ]:
importlib.reload(psdClass)

RETURN_PSD_1sec = True

# TODO: INCLUDE ALTEARNTIVE BASELINING FOR FREE
# in plotting, but also still in STATS CREATION

(
    PSDs_free, BLs_free
) = psdClass.get_allSpecStates_Psds(
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    incl_lidmove=False,
    incl_rest=False,
    incl_tap=False,
)


Plot REST or MOVEMENTS: Voluntary (TAP) / Involuntary (LID) versus CONTRA and IPSI-LATERAL Ephys

- movement selection has millisecond resolution

In [ ]:
importlib.reload(plot_ssd_PSDs)
importlib.reload(plot_specPsd)
importlib.reload(psdClass)
importlib.reload(ephySel)
importlib.reload(prep_specStats)

# PLOT DYSKINESIA STATES

%matplotlib inline
# %matplotlib qt
INCL_STATS = True
ALT_BASELINE = False
REST_u30_BASELINE = True
ALPHA=.01
ADD_FIGNAME = '0227_a01_'

STATS_VERSION='2Hz'

for PLOT_SEL in ['REST', 'TAP', 'INVOLUNT', ]:  # 'FREE'
    
    # select state selection to plot
    print(f'PLOT {PLOT_SEL}, (incl STATs: {INCL_STATS})')
    
    # select matching PSD dict
    if PLOT_SEL == 'FREE':
        PSD_DICT = PSDs_free
        BL_DICT = BLs_free
    else:
        PSD_DICT = PSDs_1s
        BL_DICT = BLs_1s

    plot_specPsd.prep_and_plot_moveSpecPsd(
        PLOT_CONDITION=PLOT_SEL,
        PSD_DICT=PSD_DICT,
        BASELINE=BL_DICT,
        SAVE_PLOT=True,
        SHOW_PLOT=False,
        INCL_STATS=True,
        ALPHA=ALPHA,
        ALT_MOVELID_BASELINE=ALT_BASELINE,
        REST_u30_BASELINE=REST_u30_BASELINE,
        STATS_VERSION=STATS_VERSION,
        STAT_PER_LID_CAT=True,
        STAT_LID_COMPARE='categs',
        MERGE_REST_STNS=True,
        MERGED_DYSK=True,
        ADD_TO_FIG_NAME=ADD_FIGNAME,
    )


In [ ]:
vars(PSDs_1s['tapright_moderatelid']).keys()

In [ ]:
PSDs_1s.keys()

## FIG) Plot PSD: REST vs MOVEMENT

In [ ]:
import lfpecog_plotting.plot_psd_restvsmove as plotRestMovePsds
import lfpecog_features.feats_spectral_helpers as specHelp

#### Spectral Powers

In [ ]:
importlib.reload(ephySel)
importlib.reload(psdClass)

RETURN_PSD_1sec = True
FT_VERSION = 'v8'

(
    PSDs_1s, BLs_1s
) = psdClass.get_allSpecStates_Psds(
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    FT_VERSION=FT_VERSION,
    incl_free=False,
    FEATURE='POWER',
)

In [ ]:
vars(PSDs_1s['dyskmoveboth_mildlid']).keys()

In [ ]:
importlib.reload(specHelp)
importlib.reload(plotRestMovePsds)

importlib.reload(psdClass)
importlib.reload(ephySel)
importlib.reload(prep_specStats)

# PLOT DYSKINESIA STATES

%matplotlib inline
# %matplotlib qt
SOURCE = 'lfp'
IPSI_CONTRA_SPLIT = 'StnEcog4'
# BASE_METHOD = 'OFF_zscore'
BASE_METHOD = 'OFF_perc_change'
PEAK_SHIFT_GAMMA = False
SMOOTH_WIN = 3
INCL_STATS = False
STATS_VERSION='4Hz'
ALPHA=.01

ADD_FIGNAME = '0327v8_StnEcog4Binary_'

if BASE_METHOD == 'OFF_perc_change':
    ADD_FIGNAME += 'Offperc_'
elif BASE_METHOD == 'OFF_zscore':
    ADD_FIGNAME += 'OffZ_'
if PEAK_SHIFT_GAMMA: ADD_FIGNAME += 'Gshift_'
if SMOOTH_WIN > 0: ADD_FIGNAME += f'sm{SMOOTH_WIN}_'
# ADD_FIGNAME = '0309_sm2_OFFperc_'
STATS_VERSION='4Hz'

# PSD dicts
PSD_DICT = PSDs_1s
BL_DICT = BLs_1s


plotRestMovePsds.prep_and_plot_restvsmove(
    FEATURE='PSD',
    SOURCE=SOURCE,
    MOVESIDES_SPLITTED=IPSI_CONTRA_SPLIT,
    LID_BINARY=True,
    PSD_DICT=PSD_DICT,
    BASELINE=BL_DICT,
    BASE_METHOD=BASE_METHOD,  # 'OFF_zscore, perc_spectral, OFF_perc_change
    REST_u30_BASELINE=False,
    SMOOTH_WIN=SMOOTH_WIN,
    MIN_SUBS_FOR_MEAN=4,
    PEAK_SHIFT_GAMMA=PEAK_SHIFT_GAMMA,
    SAVE_PLOT=True,
    SHOW_PLOT=False,
    INCL_STATS=INCL_STATS,
    STATS_VERSION=STATS_VERSION,
    ADD_TO_FIG_NAME=ADD_FIGNAME,
)

#### Coherences (consider suppl)

In [ ]:
importlib.reload(ephySel)
importlib.reload(psdClass)

# COH_SRC = 'STNs'
COH_TYPE = 'SQCOH'

RETURN_PSD_1sec = True
COHs_1s, COH_BL = {}, {}

# for coh_ft in ['SQCOH', 'ICOH']:
#     (
#         COHs_1s[coh_ft], COH_BL[coh_ft]
#     ) = psdClass.get_allSpecStates_Psds(
#         RETURN_PSD_1sec=RETURN_PSD_1sec,
#         incl_free=False,
#         FEATURE=f'COH_{COH_SRC}',
#         COH_TYPE=coh_ft
#     )

for COH_SRC in ['STNs', 'STNECOG']:
    (
        COHs_1s[COH_SRC], COH_BL[COH_SRC]
    ) = psdClass.get_allSpecStates_Psds(
        FT_VERSION='v8',
        RETURN_PSD_1sec=RETURN_PSD_1sec,
        incl_free=False,
        FEATURE=f'COH_{COH_SRC}',
        COH_TYPE=COH_TYPE,
    )
print(COH_TYPE)

In [ ]:
importlib.reload(plotRestMovePsds)

importlib.reload(psdClass)
importlib.reload(ephySel)
importlib.reload(prep_specStats)

# PLOT DYSKINESIA STATES

%matplotlib inline
# %matplotlib qt
BASE_METHOD = 'OFF_zscore'
# BASE_METHOD = 'OFF_perc_change'
PEAK_SHIFT_GAMMA = True
SMOOTH_WIN = 3
INCL_STATS = False
ALPHA=.01

ADD_FIGNAME = '0327v8_'

if BASE_METHOD == 'OFF_perc_change':
    ADD_FIGNAME += 'Offperc_'
elif BASE_METHOD == 'OFF_zscore':
    ADD_FIGNAME += 'OffZ_'
if PEAK_SHIFT_GAMMA: ADD_FIGNAME += 'Gshift_'
if SMOOTH_WIN > 0: ADD_FIGNAME += f'sm{SMOOTH_WIN}_'
# ADD_FIGNAME = '0309_sm2_OFFperc_'
STATS_VERSION='4Hz'


# COHERENCE dicts
# COH_TYPE = 'SQCOH'
PSD_DICT = COHs_1s
BL_DICT = COH_BL

plotRestMovePsds.prep_and_plot_restvsmove(
    FEATURE=COH_TYPE,
    SOURCE=COH_SRC,
    MOVESIDES_SPLITTED='COH_4panel',
    PSD_DICT=PSD_DICT,
    BASELINE=BL_DICT,
    BASE_METHOD=BASE_METHOD,  # 'OFF_zscore, perc_spectral, OFF_perc_change
    SMOOTH_WIN=SMOOTH_WIN,
    MIN_SUBS_FOR_MEAN=4,
    PEAK_SHIFT_GAMMA=PEAK_SHIFT_GAMMA,
    SAVE_PLOT=True,
    SHOW_PLOT=False,
    INCL_STATS=INCL_STATS,
    STATS_VERSION=STATS_VERSION,
    ADD_TO_FIG_NAME=ADD_FIGNAME,
)


## FIG) Boxplots: Rest vs Move, LID binary

PSD visualization 1-SEC with boxplot

In [ ]:
import lfpecog_features.feats_spectral_features as specFeats
import lfpecog_plotting.plot_move_spec_boxes as plotMoveBoxes

In [ ]:
# execute boxplotting
importlib.reload(utilsFiles)
importlib.reload(specHelp)
importlib.reload(specFeats)
importlib.reload(plotMoveBoxes)
importlib.reload(psdClass)

FT_VERSION = 'v8'
RETURN_PSD_1sec = True

# pre load POWER values
(
    PSDs_1s, BLs_1s
) = psdClass.get_allSpecStates_Psds(
    FT_VERSION=FT_VERSION,
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    incl_free=False,
    FEATURE='POWER',
)

# get freq bands and indiv peaks
f_bands = utilsFiles.load_ft_ext_cfg(FT_VERSION=FT_VERSION)['SPECTRAL_BANDS'].copy()
f_bands['gammaBroad'] = [60, 90]
del(f_bands['gamma1'], f_bands['gamma2'], f_bands['gamma3'])

# peak_df = specHelp.get_indiv_band_peaks(SRC='lfp')
# f = peak_df.loc['(109): dysk']['narrow_gamma']
# print(f)
# if np.isnan(f):
#     f = peak_df.loc['(109): tap']['narrow_gamma']

#     print(f)




Plot only boxplots (not included Sept '24)

In [ ]:
importlib.reload(psdClass)
importlib.reload(plotMoveBoxes)
importlib.reload(plotRestMovePsds)

# execute Plot
plotMoveBoxes.plotBoxes_moveBin_LidBin(
    PSDs_1s=PSDs_1s,
    PSD_BLs_1s=BLs_1s,
    FT_VERSION='v8',
    INCL_SIGN=False,
    SAVE_FIG=True,
    FIGDATE='0910v8',
)

#### FIG) Spectral Densities and Stat-Boxplots

Get Data (COH), PSD above

In [ ]:
### INCLUDE COHERENCES

COHs_1s, COH_BL = {}, {}

for COH_SRC in ['STNs', 'STNECOG']:
    (
        COHs_1s[COH_SRC], COH_BL[COH_SRC]
    ) = psdClass.get_allSpecStates_Psds(
        FT_VERSION='v8',
        incl_free=False,
        FEATURE=f'COH_{COH_SRC}',
        COH_TYPE='SQCOH',
    )

In [ ]:
### imag COH

imagCOHs_1s, imagCOH_BL = {}, {}

for COH_SRC in ['STNs', 'STNECOG']:
    (
        imagCOHs_1s[COH_SRC], imagCOH_BL[COH_SRC]
    ) = psdClass.get_allSpecStates_Psds(
        FT_VERSION='v8',
        incl_free=False,
        FEATURE=f'COH_{COH_SRC}',
        COH_TYPE='ICOH',
    )

#### Spectra with Boxes as subplots

In [ ]:
importlib.reload(plotRestMovePsds)
importlib.reload(plotMoveBoxes)

FT = 'SQCOH'  # SQCOH / Power / ICOH

FIG_NAME = f'0910_{FT}Spectra_Boxes_ft{FT_VERSION}'
if FT == 'Power':
    ft_dict = PSDs_1s
    bl_dict = BLs_1s
    
elif FT == 'SQCOH':
    ft_dict = COHs_1s
    bl_dict = COH_BL

elif FT == 'ICOH':
    ft_dict = imagCOHs_1s
    bl_dict = imagCOH_BL

plotMoveBoxes.plotPsdrestmove_boxStats(
    FT=FT, FIG_NAME=FIG_NAME,
    ft_dict=ft_dict, bl_dict=bl_dict,
    INCL_BOX_SIGN=True,
    INCL_SIG_REST30=True,
)

Get and save indiv peak frequencies

In [ ]:
importlib.reload(ftHelpers)

# save indiv peak data frame as excel for readable output
SRC = 'ecog'  # ecog

peak_xcl = os.path.join(get_project_path('results'),
                        'features', 'SSD_feats_broad_v6', 'v4.0',
                        f'indiv_peak_df_{SRC}.xlsx')

# calculate indiv peak freqs for different clinical states
PEAK_DICT = {
    'rest': ftHelpers.get_indiv_peak_freqs(psd_dict=PSDs_1s, STATE='rest', SOURCE_SEL=SRC,),
    'dysk': ftHelpers.get_indiv_peak_freqs(psd_dict=PSDs_1s, STATE='dyskmove', SOURCE_SEL=SRC,),
    'tap': ftHelpers.get_indiv_peak_freqs(psd_dict=PSDs_1s, STATE='tap', SOURCE_SEL=SRC,),
    'all': ftHelpers.get_indiv_peak_freqs(psd_dict=PSDs_1s, STATE='all', SOURCE_SEL=SRC,)
}

peak_df = pd.DataFrame(columns=['lo_beta', 'hi_beta', 'narrow_gamma'])

for sub in sorted(SUBS):
    
    peak_df.loc[f'SUB-{sub}'] = [np.nan] * 3
    
    for s in PEAK_DICT.keys():
        # print('\t', s, PEAK_DICT[s][sub])
        peaks = PEAK_DICT[s][sub]
        if len(peaks) == 0:
            peak_df.loc[f'({sub}): {s}'] = [np.nan] * 3
            continue
        peak_df.loc[f'({sub}): {s}'] = [peaks[c] for c in peak_df.keys()]
    
peak_df.to_excel(peak_xcl)

In [ ]:
import lfpecog_plotting.plot_move_spec_boxes as plotSpecBoxes

#### Old) Boxes between for Spectral changes for Rest OR Movement (vs. LID)



In [ ]:
importlib.reload(psd_Stats)
importlib.reload(psdClass)
importlib.reload(plotSpecBoxes)

%matplotlib inline

# STATE_SEL = 'lfp'
# STATE_SEL = 'movement'
# SOURCE_SEL = 'rest'
EPOCH_LEN = 1
LOG=False

for STATE_SEL, SOURCE_SEL in product(
    ['rest', 'movement'], ['lfp', 'ecog']
):
    # if not (STATE_SEL == 'movement' and SOURCE_SEL == 'lfp'): continue

    psd_box, ids_box = plotSpecBoxes.calc_fband_boxlists(
        psd_dict=PSDs_1s, baselines=BLs_1s,
        STATE_SEL=STATE_SEL,
        MEAN_EPOCH=EPOCH_LEN,  # take mean averages as input for (in the end LMM)
        SRC_SEL=SOURCE_SEL,
        verbose=False,
    )
    FIG_END = f'_epoch{EPOCH_LEN}'
    # if LOG: FIG_END += '_log'
    plotSpecBoxes.plot_moveSpec_boxplots(
        psd_box=psd_box,
        ids_box=ids_box,
        STATE_SEL=STATE_SEL,
        SOURCE_SEL=SOURCE_SEL,
        ADD_SUB_DOTS=True,
        ADD_LMM_COEF=True,
        FIG_SAVE=True,
        fig_name_start=f'2302sq_',
        fig_add_end=FIG_END,
        SQUEEZE_FIG=True,
    )


## FIG) Plot Violins between Voluntary and Involuntary Movement (vs. LID)

In [ ]:
importlib.reload(plotSpecBoxes)

psd_box, ids_box, movetypes = plotSpecBoxes.calc_fband_boxlists(
    psd_dict=PSDs_1s, baselines=BLs_1s,
    STATE_SEL='dyskmove',  # movement
    SPLIT_MOVEMENT=False,
    SPLIT_CONTRAIPSI=True,
    MEAN_EPOCH=EPOCH_LEN,  # take mean averages as input for (in the end LMM)
    SRC_SEL='lfp',
    verbose=False,
)

In [ ]:
importlib.reload(plotSpecBoxes)

# TODO: FIX UNILAT IPSI CONTRA

SOURCE = 'lfp'  # lfp ecog
EPOCH_LEN = 1

psd_box, ids_box, movetypes = plotSpecBoxes.calc_fband_boxlists(
    psd_dict=PSDs_1s, baselines=BLs_1s,
    STATE_SEL='dyskmove',  # movement
    SPLIT_MOVEMENT=False,
    SPLIT_CONTRAIPSI=True,
    MEAN_EPOCH=EPOCH_LEN,  # take mean averages as input for (in the end LMM)
    SRC_SEL=SOURCE,
    verbose=True,
)

plotSpecBoxes.plot_moveSplit_violins(
    psd_box=psd_box,
    ids_box=ids_box,
    movetypes=movetypes,
    SOURCE_SEL=SOURCE,
    ADD_SUB_DOTS=False,
    ADD_STAT='LMM',  # 'LMM', 'MWU', False
    FIG_SAVE=True,
    fig_name_start=f'2602_epoch{EPOCH_LEN}_onlyDysk',
)

### 2 B) Coherence visualization, 1 sec epochs for selected movement/rest

In [ ]:
vars(PSDs_1s['rest_mildlid']).keys()

### 2 C) load dataclass with 10 SEC-features, labels, and acc

In [ ]:

# importlib.reload(ftProc)

# class created in ftProc, FeatLidClass()
# mean-RMS standardly zscored
import lfpecog_analysis.ft_processing_helpers as ftProc

featLabPath = os.path.join(utilsFiles.get_project_path('data'),
                           'prediction_data',
                           'featLabelClasses')
feats6 = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, 'featLabels_ftv6_Cdrs_StnOnly.P'),
    convert_float_np64=True
)

In [ ]:
clrs = list(plotHelp.get_colors('PaulTol').values())

### bis) Develop and Visualise Movement State Detection


#### Run single Acc-State Detections


In [ ]:
importlib.reload(run_tap_detect)
importlib.reload(movePrep)
importlib.reload(findTap)

taplists = {}
for sub in ['012',  '014']:  # '008', '013',
    print(sub)
    taplists[sub] = run_tap_detect.runTapDetection(subData[sub])

#### Visualise Performance of Tap/Move-detection

In [ ]:

fonts=20

for sub in list(subData.keys()):

    for x0, x1 in zip(
        # [9, 42],
        # [10, 43]
        [5, 37,],
        [15, 42]
    ):

        fig, axes = plt.subplots(2, 1, figsize=(16, 8))

        for s, side in enumerate(['left', 'right']):

            acc_df = getattr(subData[sub], f'acc_{side}').data  # per side
            fs = getattr(subData[sub], f'acc_{side}').fs

            ax = movePrep.find_main_axis(
                acc_df.iloc[:, 1:4].values
            )
            svm = movePrep.signalvectormagn(
                acc_df.iloc[:, 1:4].values
            )

            axes[s].plot(
                acc_df['dopa_time'] / 60,
                acc_df.iloc[:, ax + 1],
                alpha=.4, label='uni-axis'
            )
            axes[s].plot(
                acc_df['dopa_time'] / 60,
                movePrep.signalvectormagn(
                    acc_df.iloc[:, 1:4].values
                ), alpha=.4, label='svm', c='r', ls='dotted',
            )

            axes[s].scatter(
                np.array([l[0] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.65e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='g', label='tap-start',
            )
            axes[s].scatter(
                np.array([l[-1] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.6e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='r', label='tap-end'
            )
            axes[s].scatter(
                np.array([m[0] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.55e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='orange', label='move-start'
            )
            axes[s].scatter(
                np.array([m[1] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.5e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='purple', label='move-end'
            )

            axes[s].set_xlim(x0, x1)
            axes[s].set_ylim(-1e-6, 1e-6)
            axes[s].set_ylabel(
                f'Acceleration\n{side.upper()}'
                    '\n(g, m/s/s)',
                size=fonts
            )
            axes[s].tick_params(labelsize=fonts - 4)

        axes[s].set_xlabel('Time (minutes to L-Dopa intake)', size=fonts)


        plt.suptitle(
            f'Subject {sub} -  bilateral '
            'Movement detection',
            x=.1, y=.96, ha='left',
            size=fonts+4
        )
        # remove duplicate legend labels
        handles, labels = plotHelp.remove_duplicate_legend(
            plt.gca().get_legend_handles_labels()
        )

        fig.legend(
            handles, labels,
            frameon=False, fontsize=fonts - 4, ncol=3,
            loc='center left', bbox_to_anchor = [.55, .95])
        
        plt.tight_layout()

        # plt.savefig(os.path.join(
        #     fig_dir, 'tapping_detection',
        #     f'sub{sub}_moveDetect_newBorders_min{x0}_{x1}'
        # ), dpi=150, facecolor='w',)

        plt.close()